In [ ]:
import pickle

# import packages

import numpy as np
from numpy import random
from scipy import stats
import pandas as pd
import copy
import matplotlib.pyplot as plt

# import r packages if needed

import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages

rpackages.importr("clusterGeneration")
rpackages.importr("mpower")
rpackages.importr("base")
rpackages.importr("utils")
cluster_generation = robjects.packages.importr("clusterGeneration")
mpower = robjects.packages.importr("mpower")
base = robjects.packages.importr("base")
utils = robjects.packages.importr("utils")

# set seed from R?
r = robjects.r
set_seed = r('set.seed')
from numpy import load


In [ ]:
# todos
# TODO 1: decide which parameters you include in specific functions (e.g. for beta confounded ex), check
# TODO 2: CLEAN CODE !
# TODO 3: Check if it makes sense to change betaconfoundedness ?

In [ ]:
# seeds for reproducibility --> NICE :-D
set_seed(8953)
random.seed(8953)

In [ ]:
# SIMULATION SETUP: "META-PARAMETERS"

sample_sizes = [1000, 2000, 5000, 10000]
test_size = 1000
runs = 30
n_setups = 28

In [ ]:
 # Fully Synthetic Data Sets (Set up from Künzel et al.)

# 1: Simulate the d-dimensional X.
# 2: Create Potential Outcomes Y(1) and Y(0).
# 3: Simulate Treatment Assignments trough W.

In [ ]:
########################################################

In [ ]:
# 1: Simulate the d-dimensional X

# Setup parameters
d = 25
N = 1000

# X Correlation matrix and mean
mean = np.zeros(d)
cov = np.array(mpower.cvine(d=d, alpha=0.5, beta=0.5))  # TODO: set cov according to setup

# Simulate X
X = random.multivariate_normal(mean=mean, cov=cov, size=N, check_valid='warn')

In [ ]:
X  # reproducibility: first value = 0.761312

In [ ]:
# 2: Create Potential Outcomes Y(1) and Y(0).

# 2.1 Simulate errors
e_0 = random.normal(loc=0.0, scale=1.0, size=N)
e_1 = random.normal(loc=0.0, scale=1.0, size=N)

# TODO: if needed take student_t distributed errors to see simulation with heavy-tailed errors

# e_o = random.standard_t(df=1,size=N)
# e_1 = random.standard_t(df=1,size=N)

In [ ]:
#######################################################

In [ ]:
# TODO: Chaos here --> Clean

In [ ]:
# 2.2 Create Response Functions

In [ ]:
# function
def simple_indicator_cate(x_train, x_test):
    betas = random.uniform(low=-5, high=5, size=x_train.shape[1])

    # train
    mu_0 = np.matmul(x_train, betas) + 5 * np.int8(x_train[:, 0] > 0.5)
    mu_1 = mu_0 + 8 * np.int8(x_train[:, 1] > 0.1)  # indicator
    tau = mu_1 - mu_0

    # test
    mu_0_test = np.matmul(x_test, betas) + 5 * np.int8(x_test[:, 0] > 0.5)
    mu_1_test = mu_0_test + 8 * np.int8(x_test[:, 1] > 0.1)
    tau_test = mu_1_test - mu_0_test

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test


In [ ]:
a, b, c, d, e, f, g = simple_indicator_cate(X, X)

In [ ]:
# SI1.1 simple cate - indicator  (no confounding)
betas_0 = random.uniform(low=-5, high=5, size=d)
mu_0 = np.matmul(X, betas_0) + 5 * np.int8(X[:, 0] > 0.5)
mu_1 = mu_0 + 8 * np.int8(X[:, 1] > 0.1)  # indicator
tau = mu_1 - mu_0
tau

In [ ]:
# function
def simple_linear_cate(x_train, x_test):
    betas = random.uniform(low=-5, high=5, size=x_train.shape[1])

    # train
    mu_0 = np.matmul(x_train, betas) + 5 * x_train[:, 0]
    mu_1 = mu_0 + 8 * x_train[:, 1]
    tau = mu_1 - mu_0

    # test
    mu_0_test = np.matmul(x_test, betas) + 5 * x_test[:, 0]
    mu_1_test = mu_0_test + 8 * x_test[:, 1]
    tau_test = mu_1_test - mu_0_test

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test

In [ ]:
# SI1.2 simple cate - linear (no confounding)
betas_0 = random.uniform(low=-5, high=5, size=d)
mu_0 = np.matmul(X, betas_0) + 5 * X[:, 0]
mu_1 = mu_0 + 8 * X[:, 1]  # linear
tau = mu_1 - mu_0
tau

In [ ]:
# function # TODO: what does betas???
def simple_quadratic_cate(x_train, x_test):
    betas = random.uniform(low=-5, high=5, size=x_train.shape[1])

    # train
    mu_0 = 2 * x_train[:, 0] - 1
    mu_1 = mu_0 + 2 * (x_train[:, 1] ** 2)
    tau = mu_1 - mu_0

    # test
    mu_0_test = 2 * x_test[:, 0] - 1
    mu_1_test = mu_0_test + 2 * (x_test[:, 1] ** 2)
    tau_test = mu_1_test - mu_0_test

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test

In [ ]:
# SI1.3 simple cate - quadratic (no confounding)
betas_0 = random.uniform(low=-5, high=5, size=d)
mu_0 = np.matmul(X, betas_0) + 5 * (X[:, 0] ** 2)
mu_1 = mu_0 + 8 * (X[:, 1] ** 2)  # quadratic
tau = mu_1 - mu_0
tau

In [ ]:
#############################################

In [ ]:
def complex_linear_cate(x_train, x_test):
    betas_0 = random.uniform(low=-5, high=5, size=x_train.shape[1])
    betas_1 = random.uniform(low=-5, high=5, size=x_train.shape[1])

    # train
    mu_0 = np.matmul(x_train, betas_0)
    mu_1 = np.matmul(x_train, betas_1)
    tau = mu_1 - mu_0

    # test
    mu_0_test = np.matmul(x_test, betas_0)
    mu_1_test = np.matmul(x_test, betas_1)
    tau_test = mu_1_test - mu_0_test

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test

In [ ]:
# SI2 complex linear cate  (in Künzel it is low=1, high=30)
betas_0 = random.uniform(low=-5, high=5, size=d)
betas_1 = random.uniform(low=-5, high=5, size=d)
mu_0 = np.matmul(X, betas_0)
mu_1 = np.matmul(X, betas_1)
tau = mu_1 - mu_0

In [ ]:
###############################################

Lahn das Laufe

In [ ]:
def varsigma_funct(x):
    return 2 / (1 + np.exp(-12 * (x - 1 / 2)))

In [ ]:
def complex_nonlinear_cate(x_train, x_test):
    # train
    mu_0 = -1 / 2 * varsigma_funct(x=x_train[:, 0]) * varsigma_funct(x=x_train[:, 1])
    mu_1 = 1 / 2 * varsigma_funct(x=x_train[:, 0]) * varsigma_funct(x=x_train[:, 1])
    tau = mu_1 - mu_0

    # test
    mu_0_test = 2 * x_test[:, 0] - 1
    mu_1_test = mu_0_test + 1 / 2 * varsigma_funct(x=x_test[:, 0]) * varsigma_funct(x=x_test[:, 1])
    tau_test = mu_1_test - mu_0_test

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test

In [ ]:
# SI3 complex non-linear
mu_0 = -1 / 2 * varsigma_funct(x=X[:, 0]) * varsigma_funct(x=X[:, 1])
mu_1 = 1 / 2 * varsigma_funct(x=X[:, 0]) * varsigma_funct(x=X[:, 1])
tau = mu_1 - mu_0

In [ ]:
#############################################

In [ ]:
def global_linear_response(x_train, x_test):
    betas = random.uniform(low=-5, high=5, size=x_train.shape[1])

    # train
    mu_0 = np.matmul(x_train, betas)
    mu_1 = mu_0
    tau = np.zeros(len(x_train))

    # test
    mu_0_test = np.matmul(x_test, betas)
    mu_1_test = mu_0_test
    tau_test = np.zeros(len(x_test))

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test

In [ ]:
# SI4 no treatment effect (global linear response)
betas = random.uniform(low=-5, high=5, size=d)
mu_0 = np.matmul(X, betas)
mu_1 = mu_0
tau = np.zeros(N)

In [ ]:
###########################################

In [ ]:
# helper function for creating piecewise beta vector
def piecewise_beta(d):
    betas = np.random.uniform(low=-5, high=5, size=d)

    betas_l = copy.copy(betas)  # betas_lower
    betas_l[5:d] = 0

    betas_m = copy.copy(betas)  #betas_middle
    betas_m[0:5] = 0
    betas_m[10:d] = 0

    betas_u = copy.copy(betas)  #betas_upper
    betas_u[0:10] = 0
    betas_u[15:d] = 0

    return betas_l, betas_m, betas_u


In [ ]:
# helper function
def piecewise_linear_new(x):
    condition_l = x[:, 19] < -0.4
    condition_u = x[:, 19] > 0.4

    array = np.zeros(len(x))
    array[condition_l] = np.matmul(x[condition_l, :], betas_l)
    array[~condition_l & ~condition_u] = np.matmul(x[~condition_l & ~condition_u, :], betas_m)
    array[condition_u] = np.matmul(x[condition_u, :], betas_u)

    return array

In [ ]:
def piecewise_linear_response(x_train, x_test):
    betas_l, betas_m, betas_u = piecewise_beta(x_train.shape[1])

    # train
    mu_0 = piecewise_linear_new(x_train)
    mu_1 = mu_0
    tau = np.zeros(len(x_train))

    # test
    mu_0_test = piecewise_linear_new(x_test)
    mu_1_test = mu_0_test
    tau_test = np.zeros(len(x_test))

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test

In [ ]:
# SI5 no treatment effect (piecewise linear response)
betas = np.random.uniform(low=-5, high=5, size=d)

betas_l = copy.copy(betas)  # betas_lower
betas_l[5:d] = 0

betas_m = copy.copy(betas)  #betas_middle
betas_m[0:5] = 0
betas_m[10:d] = 0

betas_u = copy.copy(betas)  #betas_upper
betas_u[0:10] = 0
betas_u[15:d] = 0


def piecewise_linear_new(x):
    condition_l = x[:, 19] < -0.4
    condition_u = x[:, 19] > 0.4

    array = np.zeros(N)
    array[condition_l] = np.matmul(x[condition_l, :], betas_l)
    array[~condition_l & ~condition_u] = np.matmul(x[~condition_l & ~condition_u, :], betas_m)
    array[condition_u] = np.matmul(x[condition_u, :], betas_u)

    return array


mu_0 = piecewise_linear_new(X)

mu_1 = mu_0

tau = np.zeros(N)

# TODO: CHECK IF RIGHT

Run these two

In [ ]:
# SI5 no treatment effect (piecewise linear response)
betas = np.random.uniform(low=-5, high=5, size=d)

betas_l = copy.copy(betas)  # betas_lower
betas_l[5:d] = 0

betas_m = copy.copy(betas)  #betas_middle
betas_m[0:5] = 0
betas_m[10:d] = 0

betas_u = copy.copy(betas)  #betas_upper
betas_u[0:10] = 0
betas_u[15:d] = 0

In [ ]:
def piecewise_linear_new(x):
    condition_l = x[:, 19] < -0.4
    condition_u = x[:, 19] > 0.4

    array = np.zeros(len(x))
    array[condition_l] = np.matmul(x[condition_l, :], betas_l)
    array[~condition_l & ~condition_u] = np.matmul(x[~condition_l & ~condition_u, :], betas_m)
    array[condition_u] = np.matmul(x[condition_u, :], betas_u)

    return array

In [ ]:
##########################################

In [ ]:
# SI6.1 beta confounded, no treatment effect # TODO: CHECK WHETER WHEN CONFOUNDED IT MAKES SENSE TO TAKE ALL BETAS, OR JUST LIKE HERE?
X = random.uniform(low=0, high=1, size=(N, d))  # ACHTUNG: nöd wiederhole!
mu_0 = 2 * X[:, 0] - 1
mu_1 = mu_0
tau = np.zeros(N)


In [ ]:
# SI6.2 beta confounded, simple cate (indicator)
X = random.uniform(low=0, high=1, size=(N, d))
mu_0 = 2 * X[:, 0] - 1
mu_1 = mu_0 + 2 * np.int8(X[:, 1] > 0.4)
tau = mu_1 - mu_0
tau

In [ ]:
# SI6.3 beta confounded, simple cate (linear)
X = random.uniform(low=0, high=1, size=(N, d))
mu_0 = 2 * X[:, 0] - 1
mu_1 = mu_0 + 2 * X[:, 1]
tau = mu_1 - mu_0


In [ ]:
# SI6.4 beta confounded, simple cate (quadratic)
X = random.uniform(low=0, high=1, size=(N, d))
mu_0 = 2 * X[:, 0] - 1
mu_1 = mu_0 + 2 * (X[:, 1] ** 2)
tau = mu_1 - mu_0
tau

In [ ]:
##################################

In [ ]:
# SI7.1 beta confounded, complex cate (linear) # TODO: SAME CHECK FOR CONFOUNDED AND BETAS!
betas = random.uniform(low=-15, high=15, size=d)

mu_0 = 2 * X[:, 0] - 1
mu_1 = mu_0 + np.matmul(X, betas)
tau = mu_1 - mu_0

# TODO: CHECK WHETHER THIS SETUP MAKES SENSE

In [ ]:
# SI7.2 beta confounded, complex cate (non-linear)
mu_0 = 2 * X[:, 0] - 1
mu_1 = mu_0 + 1 / 2 * varsigma_funct(x=X[:, 0]) * varsigma_funct(x=X[:, 1])

# TODO: CHECK WHETER THIS SETUP MAKES SENSE

Fixed parts

In [ ]:
###################################
# 2.3 Creat Potential Outcomes, FIX
Y_0 = mu_0 + e_0
Y_1 = mu_1 + e_1

In [ ]:
# quick check Y_0
Y_0

In [ ]:
# Quick Check Y_1
Y_1

In [ ]:
###################################
# 3.1 Propensity score setups # TODO: change for setup

# i) constant, balanced
e_x = 0.5

# ii) constant, unbalanced
e_x = 0.01

In [ ]:
#################################

In [ ]:
# helper function
def beta_balanced(x_train, x_test):
    beta_dist = stats.beta(a=2, b=4)  # set beta distribution

    # train
    cdf_values = stats.norm.cdf(x_train[:, 0])
    beta_values = beta_dist.pdf(cdf_values)  # calculate pdf values for x1
    e_x = 1 / 4 * (1 + beta_values)

    # test
    cdf_values_test = stats.norm.cdf(x_test[:, 0])
    beta_values_test = beta_dist.pdf(cdf_values_test)  # calculate pdf values for x1
    e_x_test = 1 / 4 * (1 + beta_values_test)

    return e_x, e_x_test


In [ ]:
# iii) beta confounded (balanced)
beta_dist = stats.beta(a=2, b=4)  # set beta distribution
cdf_values = stats.norm.cdf(X[:, 0])
beta_values = beta_dist.pdf(cdf_values)  # calculate pdf values for x1
e_x = 1 / 4 * (1 + beta_values)

pd.DataFrame(e_x).describe()  # summary stats of e_x

In [ ]:
pd.DataFrame(e_x).hist(bins=20)

In [ ]:
# check if balanced
W = random.binomial(size=N, n=1, p=e_x)
pd.DataFrame(W).describe()

In [ ]:
#####################################

In [ ]:
def beta_unbalanced(x_train, x_test):
    beta_dist = stats.beta(a=2, b=4)  # set beta distribution

    # train
    cdf_values = stats.norm.cdf(x_train[:, 0])
    beta_values = beta_dist.pdf(cdf_values)  # calculate pdf values for x1
    e_x = 1 / 40 * (1 + beta_values)

    # test
    cdf_values_test = stats.norm.cdf(x_test[:, 0])
    beta_values_test = beta_dist.pdf(cdf_values_test)  # calculate pdf values for x1
    e_x_test = 1 / 40 * (1 + beta_values_test)

    return e_x, e_x_test

In [ ]:
# iv) beta confounded (unbalanced)
beta_dist = stats.beta(a=2, b=4)  # set beta distribution
cdf_values = stats.norm.cdf(X[:, 0])
beta_values = beta_dist.pdf(cdf_values)  # calculate pdf values for x1
e_x = 1 / 20 * (1 + beta_values)

pd.DataFrame(e_x).describe()  # summary stats of e_x

In [ ]:
pd.DataFrame(e_x).hist(bins=20)  # summary stats of e_x

In [ ]:
# check if unbalanced and how much
W = random.binomial(size=N, n=1, p=e_x)
pd.DataFrame(W).describe()

In [ ]:
beta_dist = stats.beta(a=2, b=4)  # set beta distribution
cdf_values = stats.norm.cdf(X[:, 0])
beta_values = beta_dist.pdf(cdf_values)  # calculate pdf values for x1
e_x = 1 / 4 * (1 + beta_values)

pd.DataFrame(e_x).describe()  # summary stats of e_x

In [ ]:
pd.DataFrame(e_x).hist(bins=20)  # summary stats of e_x

In [ ]:
W = random.binomial(size=N, n=1, p=e_x)
pd.DataFrame(W).describe()

In [ ]:
##########################################

In [ ]:
X

In [ ]:
###################################

In [ ]:
# 3.2 Simulate Treatment Assignments trough W

# Simulate Treatment Assignment, FIX
W = random.binomial(size=N, n=1, p=e_x)

# Create Observed Outcome, FIX
ones = np.ones(N)
Y = np.multiply(W, Y_1) + np.multiply(ones - W, Y_0)

In [ ]:
pd.DataFrame(e_x).describe()  # summary stats of e_x

In [ ]:
# check out observed outcomes
Y

In [ ]:
pd.DataFrame(Y).describe()  # summary stats of Y

In [ ]:
X

In [ ]:
Y

In [ ]:
W

In [ ]:
tau

In [ ]:
dataset = np.concatenate((np.reshape(Y, (N, 1)), X, np.reshape(W, (N, 1)), np.reshape(tau, (N, 1))), axis=1)
dataset

In [ ]:
data.apppend(dataset)

# Helper function to generate datasets

In [ ]:
# SIMULATION SETUP: "META-PARAMETERS"

sample_sizes = [1000, 2000, 5000, 10000]
test_size = 1000
n_runs = 30
n_setups = 28
d = 25

In [ ]:
# all settings of e_x
exs = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
       'beta_balanced', 'beta_balanced', 'beta_balanced', 'beta_balanced', 'beta_balanced', 'beta_balanced',
       'beta_balanced',
       'beta_unbalanced', 'beta_unbalanced', 'beta_unbalanced', 'beta_unbalanced', 'beta_unbalanced', 'beta_unbalanced',
       'beta_unbalanced']

In [ ]:
# all settings of mu_0/mu_1
cates = ['glinear', 'plinear', 'sindcate', 'slinearcate', 'squadcate', 'clinearcate', 'cnonlincate'] * 4

In [ ]:
# TODO: TRIPPLE CHECK THIS FUNCTION, IT IS VERY IMPORTANT
def generate(ex, cate, sample_size, test_size):  # TODO: add test set

    # 1 generated x_train & x_test
    cov = np.array(mpower.cvine(d=d, alpha=0.5, beta=0.5))  # TODO: set cov according to setup
    X_train = random.multivariate_normal(mean=mean, cov=cov, size=sample_size, check_valid='warn')
    X_test = random.multivariate_normal(mean=mean, cov=cov, size=test_size, check_valid='warn')

    # 2 generate e_0 & e_0 (TODO: add test?)
    # train
    e_0 = random.normal(loc=0.0, scale=1.0, size=sample_size)
    e_1 = random.normal(loc=0.0, scale=1.0, size=sample_size)

    # test
    e_0_test = random.normal(loc=0.0, scale=1.0, size=test_size)
    e_1_test = random.normal(loc=0.0, scale=1.0, size=test_size)

    # 3 compute mu_0 & mu_1 --> based on setting
    if cate == 'glinear':  # 'global linear response' setting (no treatment effect)
        betas = random.uniform(low=-5, high=5, size=d)

        # train
        mu_0 = np.matmul(X_train, betas)
        mu_1 = mu_0
        tau = np.zeros(sample_size)

        # test
        mu_0_test = np.matmul(X_test, betas)
        mu_1_test = mu_0_test
        tau_test = np.zeros(test_size)


    elif cate == 'plinear':  # 'piecewise linear response' setting (no treatment effect)
        betas = np.random.uniform(low=-5, high=5, size=d)

        betas_l = copy.copy(betas)  # betas_lower
        betas_l[5:d] = 0

        betas_m = copy.copy(betas)  #betas_middle
        betas_m[0:5] = 0
        betas_m[10:d] = 0

        betas_u = copy.copy(betas)  #betas_upper
        betas_u[0:10] = 0
        betas_u[15:d] = 0

        # train
        mu_0 = piecewise_linear_new(X_train)
        mu_1 = mu_0
        tau = np.zeros(sample_size)

        # test
        mu_0_test = piecewise_linear_new(X_test)
        mu_1_test = mu_0_test
        tau_test = np.zeros(test_size)


    elif cate == 'sindcate':  # 'simple indicator cate' setting
        betas_0 = random.uniform(low=-5, high=5, size=d)

        # train
        mu_0 = np.matmul(X_train, betas_0) + 5 * np.int8(X_train[:, 0] > 0.5)
        mu_1 = mu_0 + 8 * np.int8(X_train[:, 1] > 0.1)
        tau = mu_1 - mu_0

        # test
        mu_0_test = np.matmul(X_test, betas_0) + 5 * np.int8(X_test[:, 0] > 0.5)
        mu_1_test = mu_0_test + 8 * np.int8(X_test[:, 1] > 0.1)
        tau_test = mu_1_test - mu_0_test

    elif cate == 'slinearcate':  # 'simple linear cate' setting
        betas_0 = random.uniform(low=-5, high=5, size=d)

        # train
        mu_0 = np.matmul(X_train, betas_0) + 5 * X_train[:, 0]
        mu_1 = mu_0 + 8 * X_train[:, 1]
        tau = mu_1 - mu_0

        # test
        mu_0_test = np.matmul(X_test, betas_0) + 5 * X_test[:, 0]
        mu_1_test = mu_0_test + 8 * X_test[:, 1]
        tau_test = mu_1_test - mu_0_test

    elif cate == 'squadcate':  # 'simple quadratic cate' setting
        betas_0 = random.uniform(low=-5, high=5, size=d)  # TODO: CHANGE THIS SETTING ? AKA ADD THE BETAS?

        # train
        mu_0 = 2 * X_train[:, 0] - 1
        mu_1 = mu_0 + 2 * (X_train[:, 1] ** 2)
        tau = mu_1 - mu_0

        # test
        mu_0_test = 2 * X_test[:, 0] - 1
        mu_1_test = mu_0_test + 2 * (X_test[:, 1] ** 2)
        tau_test = mu_1_test - mu_0_test

    elif cate == 'clinearcate':  # 'complex linear cate' setting
        betas_0 = random.uniform(low=-5, high=5, size=d)
        betas_1 = random.uniform(low=-5, high=5, size=d)

        # train
        mu_0 = np.matmul(X_train, betas_0)
        mu_1 = np.matmul(X_train, betas_1)
        tau = mu_1 - mu_0

        # test
        mu_0_test = np.matmul(X_test, betas_0)
        mu_1_test = np.matmul(X_test, betas_1)
        tau_test = mu_1_test - mu_0_test

    elif cate == 'cnonlincate':  # 'complex non-linear cate' setting
        # train
        mu_0 = 2 * X_train[:, 0] - 1
        mu_1 = mu_0 + 1 / 2 * varsigma_funct(x=X_train[:, 0]) * varsigma_funct(x=X_train[:, 1])
        tau = mu_1 - mu_0

        # test
        mu_0_test = 2 * X_test[:, 0] - 1
        mu_1_test = mu_0_test + 1 / 2 * varsigma_funct(x=X_test[:, 0]) * varsigma_funct(x=X_test[:, 1])
        tau_test = mu_1_test - mu_0_test

    else:
        raise (NotImplementedError)

    # 4 create potential outcomes Y_0 & Y_1 (TODO: test set?)
    # train
    Y_0 = mu_0 + e_0
    Y_1 = mu_1 + e_1

    # test
    Y_0_test = mu_0_test + e_0_test
    Y_1_test = mu_1_test + e_1_test

    # 5 SET PROPENSITY SCORE E_X --> based on setting
    if isinstance(ex, float):
        e_x = ex
        e_x_test = ex

    elif ex == 'beta_balanced':
        beta_dist = stats.beta(a=2, b=4)  # set beta distribution

        # train
        cdf_values = stats.norm.cdf(X_train[:, 0])
        beta_values = beta_dist.pdf(cdf_values)  # calculate pdf values for x1
        e_x = 1 / 4 * (1 + beta_values)

        # test
        cdf_values_test = stats.norm.cdf(X_test[:, 0])
        beta_values_test = beta_dist.pdf(cdf_values_test)  # calculate pdf values for x1
        e_x_test = 1 / 4 * (1 + beta_values_test)

    elif ex == 'beta_unbalanced':
        beta_dist = stats.beta(a=2, b=4)  # set beta distribution

        #train
        cdf_values = stats.norm.cdf(X_train[:, 0])
        beta_values = beta_dist.pdf(cdf_values)  # calculate pdf values for x1
        e_x = 1 / 20 * (1 + beta_values)

        # test
        #train
        cdf_values_test = stats.norm.cdf(X_test[:, 0])
        beta_values_test = beta_dist.pdf(cdf_values_test)  # calculate pdf values for x1
        e_x_test = 1 / 20 * (1 + beta_values_test)

    # 6 generate treatment assignment  W
    W = random.binomial(size=sample_size, n=1, p=e_x)
    W_test = random.binomial(size=test_size, n=1, p=e_x_test)

    # 7 create observed variables Y
    Y = np.multiply(W, Y_1) + np.multiply(np.ones(sample_size) - W, Y_0)
    Y_test = np.multiply(W_test, Y_1_test) + np.multiply(np.ones(test_size) - W_test, Y_0_test)

    # 8 create train & test sets
    dataset_train = np.concatenate(
        (np.reshape(Y, (sample_size, 1)), X_train, np.reshape(W, (sample_size, 1)), np.reshape(tau, (sample_size, 1))),
        axis=1)
    dataset_test = np.concatenate((np.reshape(Y_test, (test_size, 1)), X_test, np.reshape(W_test, (test_size, 1)),
                                   np.reshape(tau_test, (test_size, 1))), axis=1)

    # 9 return both sets
    return [dataset_train, dataset_test]


In [ ]:
# def generate(): --> für ein dataset!

# loop over setups (28)
## loop over sample_sizes (4)
### loop over runs (30)


# data[0][0][0][0]  # data[setup][sample_size][run][train/test]


generate('beta_unbalanced', 'cnonlincate', 1000, 500)[1]

# NICE :)


In [ ]:
# create empty list to save all datasets
data = []
for i in range(n_setups):
    data.append([])

for i in range(n_setups):
    for s in range(4):
        data[i].append([])

for i in range(n_setups):
    for s in range(4):
        for r in range(n_runs):
            data[i][s].append([])

for i in range(n_setups):
    for s in range(4):
        for r in range(n_runs):
            for t in range(2):
                data[i][s][r].append([])

data

In [ ]:
# generate all datasets
s = 0
for setup in range(n_setups):
    for size in sample_sizes:
        for run in range(n_runs):
            print(setup)
            gen = generate(ex=exs[setup], cate=cates[setup], sample_size=size, test_size=test_size)
            data[setup][s][run][0] = gen[0]  # add train-set
            data[setup][s][run][1] = gen[1]  # add test-set
        s += 1  # update index for sample_sizes
    s = 0  # reset index for sample_sizes

# it took 43 seconds # 4.4 GB of data!

TODO: Change the generate function such that test size is always the same, you know?

In [ ]:
data[27][3][29][1]  # NICE :)

In [ ]:
28 * 4 * 30 * 7 * 3  # 3360 datasets! 70560 trainings!!!

In [ ]:
# save data
sample_list = [1, 2, 3]
file_name = "/Users/arberimbibaj/Documents/Master Thesis ETH/DataSets /Generated/data_synthetic.pkl"

open_file = open(file_name, "wb")
pickle.dump(data, open_file)
open_file.close()

open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

print(loaded_list)

In [ ]:
file_name = "/Users/arberimbibaj/Documents/Master Thesis ETH/DataSets /Generated/data_synthetic.pkl"
open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

In [ ]:
loaded_list[0][0][0][0]

In [ ]:
# TODO: CHANGE THIS BACK TO WHAT IT WAS, SO NO CRASHES CAN HAPPEN.
# Current values:
# NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
# NotebookApp.rate_limit_window=3.0 (secs)

# FROM SCRATCH

In [ ]:
# seeds for reproducibility --> NICE :-D
set_seed(8953)
random.seed(8953)

In [ ]:
# SIMULATION SETUP: "META-PARAMETERS"
sample_sizes = [100, 200, 500, 1000]
test_size = 50
n_runs = 5
n_setups = 28
d = 20

In [ ]:
# all settings of e_x
exs = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
       'beta_balanced', 'beta_balanced', 'beta_balanced', 'beta_balanced', 'beta_balanced', 'beta_balanced',
       'beta_balanced',
       'beta_unbalanced', 'beta_unbalanced', 'beta_unbalanced', 'beta_unbalanced', 'beta_unbalanced', 'beta_unbalanced',
       'beta_unbalanced']

In [ ]:
# all settings of mu_0 / mu_1
cates = ['glinear', 'plinear', 'sindcate', 'slinearcate', 'squadcate', 'clinearcate', 'cnonlincate'] * 4

Cate Settings function

In [ ]:
def simple_indicator_cate(x_train, x_test):
    betas = random.uniform(low=-5, high=5, size=x_train.shape[1])

    # train
    mu_0 = np.matmul(x_train, betas) + 5 * np.int8(x_train[:, 0] > 0.5)
    mu_1 = mu_0 + 8 * np.int8(x_train[:, 1] > 0.1)  # indicator
    tau = mu_1 - mu_0

    # test
    mu_0_test = np.matmul(x_test, betas) + 5 * np.int8(x_test[:, 0] > 0.5)
    mu_1_test = mu_0_test + 8 * np.int8(x_test[:, 1] > 0.1)
    tau_test = mu_1_test - mu_0_test

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test


In [ ]:
def simple_linear_cate(x_train, x_test):
    betas = random.uniform(low=-5, high=5, size=x_train.shape[1])

    # train
    mu_0 = np.matmul(x_train, betas) + 5 * x_train[:, 0]
    mu_1 = mu_0 + 8 * x_train[:, 1]
    tau = mu_1 - mu_0

    # test
    mu_0_test = np.matmul(x_test, betas) + 5 * x_test[:, 0]
    mu_1_test = mu_0_test + 8 * x_test[:, 1]
    tau_test = mu_1_test - mu_0_test

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test

In [ ]:
# TODO: what does betas???
def simple_quadratic_cate(x_train, x_test):
    betas = random.uniform(low=-5, high=5, size=x_train.shape[1])

    # train
    mu_0 = 2 * x_train[:, 0] - 1
    mu_1 = mu_0 + 2 * (x_train[:, 1] ** 2)
    tau = mu_1 - mu_0

    # test
    mu_0_test = 2 * x_test[:, 0] - 1
    mu_1_test = mu_0_test + 2 * (x_test[:, 1] ** 2)
    tau_test = mu_1_test - mu_0_test

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test

In [ ]:
def complex_linear_cate(x_train, x_test):
    betas_0 = random.uniform(low=-5, high=5, size=x_train.shape[1])
    betas_1 = random.uniform(low=-5, high=5, size=x_train.shape[1])

    # train
    mu_0 = np.matmul(x_train, betas_0)
    mu_1 = np.matmul(x_train, betas_1)
    tau = mu_1 - mu_0

    # test
    mu_0_test = np.matmul(x_test, betas_0)
    mu_1_test = np.matmul(x_test, betas_1)
    tau_test = mu_1_test - mu_0_test

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test

In [ ]:
# helper function
def varsigma_funct(x):
    return 2 / (1 + np.exp(-12 * (x - 1 / 2)))

In [ ]:
def complex_nonlinear_cate(x_train, x_test):
    # train
    mu_0 = -1 / 2 * varsigma_funct(x=x_train[:, 0]) * varsigma_funct(x=x_train[:, 1])
    mu_1 = 1 / 2 * varsigma_funct(x=x_train[:, 0]) * varsigma_funct(x=x_train[:, 1])
    tau = mu_1 - mu_0

    # test
    mu_0_test = 2 * x_test[:, 0] - 1
    mu_1_test = mu_0_test + 1 / 2 * varsigma_funct(x=x_test[:, 0]) * varsigma_funct(x=x_test[:, 1])
    tau_test = mu_1_test - mu_0_test

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test

In [ ]:
def global_linear_response(x_train, x_test):
    betas = random.uniform(low=-5, high=5, size=x_train.shape[1])

    # train
    mu_0 = np.matmul(x_train, betas)
    mu_1 = mu_0
    tau = np.zeros(len(x_train))

    # test
    mu_0_test = np.matmul(x_test, betas)
    mu_1_test = mu_0_test
    tau_test = np.zeros(len(x_test))

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test

In [ ]:
# helper function
def piecewise_beta(d):
    betas = np.random.uniform(low=-5, high=5, size=d)

    betas_l = copy.copy(betas)  # betas_lower
    betas_l[5:d] = 0

    betas_m = copy.copy(betas)  #betas_middle
    betas_m[0:5] = 0
    betas_m[10:d] = 0

    betas_u = copy.copy(betas)  #betas_upper
    betas_u[0:10] = 0
    betas_u[15:d] = 0

    return betas_l, betas_m, betas_u


In [ ]:
# helper function
def piecewise_linear_new(x, betas_l, betas_u, betas_m):
    condition_l = x[:, 19] < -0.4
    condition_u = x[:, 19] > 0.4

    array = np.zeros(len(x))
    array[condition_l] = np.matmul(x[condition_l, :], betas_l)
    array[~condition_l & ~condition_u] = np.matmul(x[~condition_l & ~condition_u, :], betas_m)
    array[condition_u] = np.matmul(x[condition_u, :], betas_u)

    return array

In [ ]:
def piecewise_linear_response(x_train, x_test):
    betas_l, betas_m, betas_u = piecewise_beta(x_train.shape[1])

    # train
    mu_0 = piecewise_linear_new(x_train, betas_l, betas_u, betas_m)
    mu_1 = mu_0
    tau = np.zeros(len(x_train))

    # test
    mu_0_test = piecewise_linear_new(x_test, betas_l, betas_u, betas_m)
    mu_1_test = mu_0_test
    tau_test = np.zeros(len(x_test))

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test

ex functions

In [ ]:
# helper function
def beta_balanced(x_train, x_test):
    beta_dist = stats.beta(a=2, b=4)  # set beta distribution

    # train
    cdf_values = stats.norm.cdf(x_train[:, 0])
    beta_values = beta_dist.pdf(cdf_values)  # calculate pdf values for x1
    e_x = 1 / 4 * (1 + beta_values)

    # test
    cdf_values_test = stats.norm.cdf(x_test[:, 0])
    beta_values_test = beta_dist.pdf(cdf_values_test)  # calculate pdf values for x1
    e_x_test = 1 / 4 * (1 + beta_values_test)

    return e_x, e_x_test


In [ ]:
def beta_unbalanced(x_train, x_test):
    beta_dist = stats.beta(a=2, b=4)  # set beta distribution

    # train
    cdf_values = stats.norm.cdf(x_train[:, 0])
    beta_values = beta_dist.pdf(cdf_values)  # calculate pdf values for x1
    e_x = 1 / 40 * (1 + beta_values)

    # test
    cdf_values_test = stats.norm.cdf(x_test[:, 0])
    beta_values_test = beta_dist.pdf(cdf_values_test)  # calculate pdf values for x1
    e_x_test = 1 / 40 * (1 + beta_values_test)

    return e_x, e_x_test

generate train/test function

In [ ]:
# TODO: TRIPPLE CHECK THIS FUNCTION, IT IS VERY IMPORTANT
def generate(ex, cate, sample_size, test_size):

    # 1 generated x_train & x_test
    mean = np.zeros(d)
    cov = np.array(mpower.cvine(d=d, alpha=0.5, beta=0.5))  # TODO: set cov according to setup
    X_train = random.multivariate_normal(mean=mean, cov=cov, size=sample_size, check_valid='warn')
    X_test = random.multivariate_normal(mean=mean, cov=cov, size=test_size, check_valid='warn')

    # 2 generate e_0 & e_0
    # train
    e_0 = random.normal(loc=0.0, scale=1.0, size=sample_size)
    e_1 = random.normal(loc=0.0, scale=1.0, size=sample_size)

    # test
    e_0_test = random.normal(loc=0.0, scale=1.0, size=test_size)
    e_1_test = random.normal(loc=0.0, scale=1.0, size=test_size)

    # 3 compute mu_0 & mu_1 --> based on setting
    if cate == 'glinear':  # 'global linear response' setting (no treatment effect)
        mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test = global_linear_response(X_train, X_test)

    elif cate == 'plinear':  # 'piecewise linear response' setting (no treatment effect)
        mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test = piecewise_linear_response(X_train, X_test)

    elif cate == 'sindcate':  # 'simple indicator cate' setting
        mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test = simple_indicator_cate(X_train, X_test)

    elif cate == 'slinearcate':  # 'simple linear cate' setting
        mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test = simple_linear_cate(X_train, X_test)

    elif cate == 'squadcate':  # 'simple quadratic cate' setting
        mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test = simple_quadratic_cate(X_train, X_test)

    elif cate == 'clinearcate':  # 'complex linear cate' setting
        mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test = complex_linear_cate(X_train, X_test)

    elif cate == 'cnonlincate':  # 'complex non-linear cate' setting
        mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test = complex_nonlinear_cate(X_train, X_test)

    else:
        raise (NotImplementedError)

    # 4 create potential outcomes Y_0 & Y_1
    # train
    Y_0 = mu_0 + e_0
    Y_1 = mu_1 + e_1

    # test
    Y_0_test = mu_0_test + e_0_test
    Y_1_test = mu_1_test + e_1_test

    # 5 SET PROPENSITY SCORE E_X --> based on setting
    if isinstance(ex, float or int):
        e_x = ex
        e_x_test = ex

    elif ex == 'beta_balanced':
        e_x, e_x_test = beta_balanced(X_train, X_test)

    elif ex == 'beta_unbalanced':
        e_x, e_x_test = beta_unbalanced(X_train, X_test)

    else:
        raise (NotImplementedError)

    # 6 generate treatment assignment  W
    W = random.binomial(size=sample_size, n=1, p=e_x)
    W_test = random.binomial(size=test_size, n=1, p=e_x_test)

    # 7 create observed variables Y
    Y = np.multiply(W, Y_1) + np.multiply(np.ones(sample_size) - W, Y_0)
    Y_test = np.multiply(W_test, Y_1_test) + np.multiply(np.ones(test_size) - W_test, Y_0_test)

    # 8 create train & test sets
    dataset_train = np.concatenate(
        (np.reshape(Y, (sample_size, 1)), X_train, np.reshape(W, (sample_size, 1)), np.reshape(tau, (sample_size, 1))),
        axis=1)
    dataset_test = np.concatenate((np.reshape(Y_test, (test_size, 1)), X_test, np.reshape(W_test, (test_size, 1)),
                                   np.reshape(tau_test, (test_size, 1))), axis=1)

    # 9 return both sets
    return [dataset_train, dataset_test]


empty list

In [ ]:
# create empty list to save all datasets
data = []
for i in range(n_setups):
    data.append([])

for i in range(n_setups):
    for s in range(4):
        data[i].append([])

for i in range(n_setups):
    for s in range(4):
        for r in range(n_runs):
            data[i][s].append([])

for i in range(n_setups):
    for s in range(4):
        for r in range(n_runs):
            for t in range(2):
                data[i][s][r].append([])

generate all

In [ ]:
# generate all datasets
s = 0
for setup in range(n_setups):
    for size in sample_sizes:
        for run in range(n_runs):
            print(setup)
            gen = generate(ex=exs[setup], cate=cates[setup], sample_size=size, test_size=test_size)
            data[setup][s][run][0] = gen[0]  # add train-set
            data[setup][s][run][1] = gen[1]  # add test-set
        s += 1  # update index for sample_sizes
    s = 0  # reset index for sample_sizes

# it took 43 seconds # 4.4 GB of data!

In [ ]:
data[0][0][0][0] # :)

In [ ]:
# -1.992791, -7.838958 -> reproducibility check over :)